## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

In [2]:
def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [3]:
from datetime import datetime
import json
from observation import Ilist, Analysis
import pandas as pd
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17']
date= dates[-1]
log = {'date_irve': date, 
       'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
#      'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  84386


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 1 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  815
nombre de pdc en itinerance   :  83571


In [6]:
# arborescence des champs et nombre de valeurs différentes
il = Ilist.obj(itinerance_init)
print(il.tree())

-1: root-derived (83571)
   0 : index (83571)
      1 : contact_operateur (134)
      2 : nom_enseigne (1271)
      3 : coordonneesXY (18591)
      4 : adresse_station (17740)
      5 : id_station_itinerance (20566)
      6 : nom_station (17509)
      7 : implantation_station (5)
      8 : nbre_pdc (52)
      9 : condition_acces (2)
      10: horaires (505)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (49690)
      13: date_maj (630)
      14: last_modified (590)


-----------------------
## 4 - Bilan initial intégrité
- plus de 50 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          42543
contact_operateur - id_station_itinerance          13317
nom_enseigne - id_station_itinerance                9856
coordonneesXY - id_station_itinerance              15099
id_station_itinerance - id_pdc_itinerance           1278
nom_station - id_station_itinerance                 3953
implantation_station - id_station_itinerance        1279
nbre_pdc - id_station_itinerance                    5107
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                   12140
station_deux_roues - id_station_itinerance         14103
adresse_station - coordonneesXY                     2067

nombre d'enregistrements sans erreurs :  36590
nombre d'enregistrements avec au moins une erreur :  46981
taux d'erreur :  56  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- plus de la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de plus de 80% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  33881  soit  72  %
nombre de pdc sans doublon   :  13100


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            335
nom_enseigne - id_station_itinerance                 507
coordonneesXY - id_station_itinerance               1504
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1682
implantation_station - id_station_itinerance         202
nbre_pdc - id_station_itinerance                    1194
condition_acces - id_station_itinerance               13
horaires - id_station_itinerance                     267
station_deux_roues - id_station_itinerance           770
adresse_station - coordonneesXY                      997

nombre d'enregistrements sans erreurs :  8756
nombre d'enregistrements avec au moins une erreur :  4344


-----------------------
## 6 - Séparation doublons station - date de maj
- plus de 15% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de plus de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 33 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  468  soit  11  %
nombre de pdc sans doublon   :  3876


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             13
nom_enseigne - id_station_itinerance                 263
coordonneesXY - id_station_itinerance                909
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1396
implantation_station - id_station_itinerance          22
nbre_pdc - id_station_itinerance                    1076
condition_acces - id_station_itinerance                8
horaires - id_station_itinerance                      52
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                      830

nombre d'enregistrements sans erreurs :  227
nombre d'enregistrements avec au moins une erreur :  3649  soit  84  %


In [12]:
# structure des données résiduelles et nombre de valeurs
il = Ilist.obj(itinerance_5)
print(il.tree())

-1: root-derived (3649)
   0 : index (3649)
      1 : contact_operateur (38)
      2 : nom_enseigne (337)
      3 : coordonneesXY (1174)
      4 : adresse_station (1179)
      5 : id_station_itinerance (1148)
         13: date_maj (88)
      6 : nom_station (1731)
      7 : implantation_station (5)
      8 : nbre_pdc (22)
      9 : condition_acces (2)
      10: horaires (70)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (3649)
      14: last_modified (68)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  66


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3649
total des doublons à supprimer :  34349
nombre de pdc avec controles ok :  45573


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
il = Ilist.obj(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (45573)
   11: id_pdc_itinerance (45573)
      4 : id_station_itinerance (19184)
         1 : nom_enseigne (941)
         2 : coordonneesXY (14976)
            0 : contact_operateur (116)
            3 : adresse_station (14332)
            8 : condition_acces (2)
         5 : nom_station (15046)
         6 : implantation_station (5)
         7 : nbre_pdc (52)
         9 : horaires (440)
         10: station_deux_roues (5)
         19: paiement_autre (7)
      12: puissance_nominale (84)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (7)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (104)
      23: date_maj (468)
      24: last_modified (395)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [19]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  73  % des écarts

support@virta.global 1026
hello@powerdot.fr 618
supervision-ev.france@totalenergies.com 506
pascal.lhermitte@siege27.fr 258
web@freshmile.com 257


In [20]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1500
41  %


In [21]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [22]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3191,80164,support@virta.global,MODULO,"[4.57827600,49.4743290]",Place de la Halle,FRS08P08354,RAUCOURT ET FLABA - 57540 - Place de la Halle,Voirie,1,Accès libre,24/7,False,FRS08E57540,2023-05-31,2023-06-17T07:53:43.745000+00:00
3192,80165,support@virta.global,MODULO,"[4.38327800,49.3957250]",Place de la Halle,FRS08P08354,RAUCOURT ET FLABA - 57539 - Place de la Halle,Voirie,2,Accès libre,24/7,False,FRS08E57539,2023-05-31,2023-06-17T07:53:43.745000+00:00
3193,80166,support@virta.global,MODULO,"[4.38327800,49.3957250]",50 Route d'Hirson,FRS08P08282,MAUBERT FONTAINE -57530 - Route d'Hirson,Voirie,1,Accès libre,24/7,False,FRS08E57530,2023-05-31,2023-06-17T07:53:43.745000+00:00
3194,80167,support@virta.global,MODULO,"[4.41915300,49.6982130]",50 Route d'Hirson,FRS08P08282,MAUBERT FONTAINE - 57529 - Route d'Hirson,Voirie,2,Accès libre,24/7,False,FRS08E57529,2023-05-31,2023-06-17T07:53:43.745000+00:00
3195,80168,support@virta.global,MODULO,"[4.41915300,49.6982130]",Rue de Raucourt,FRS08P08357,REMILLY AILLICOURT - 57520 - Rue de Raucourt,Voirie,1,Accès libre,24/7,False,FRS08E57520,2023-05-31,2023-06-17T07:53:43.745000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,80309,support@virta.global,MODULO,"[4.49431027,49.7880936]",Gare de Lucquy,FRS08P08262,LUCQUY - 57626 - Gare,Voirie,1,Accès libre,24/7,False,FRS08E57626,2023-05-31,2023-06-17T07:53:43.745000+00:00
3326,80310,support@virta.global,MODULO,"[4.70237100,49.9925440]",Gare de Lucquy,FRS08P08262,LUCQUY - 57625 - Gare,Voirie,2,Accès libre,24/7,False,FRS08E57625,2023-05-31,2023-06-17T07:53:43.745000+00:00
3651,84035,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299331,2022-11-03,2023-06-17T07:51:01.116000+00:00
3652,84036,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299351,2022-11-03,2023-06-17T07:51:01.116000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [23]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1693,64108,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE205868TE,2022-07-25,2023-06-17T07:55:47.771000+00:00
1696,64219,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201426,2022-07-25,2023-06-17T07:55:47.771000+00:00
1697,64314,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200921,2022-07-25,2023-06-17T07:55:47.771000+00:00
1698,64345,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200977,2022-07-25,2023-06-17T07:55:47.771000+00:00
1699,64392,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE205868,2022-07-25,2023-06-17T07:55:47.771000+00:00
1700,64394,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201290,2022-07-25,2023-06-17T07:55:47.771000+00:00
1704,64496,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205871TE,2022-07-25,2023-06-17T07:55:46.185000+00:00
1705,64506,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE201396,2022-07-25,2023-06-17T07:55:46.185000+00:00
1706,64507,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205675,2022-07-25,2023-06-17T07:55:46.185000+00:00
1707,64508,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205871,2022-07-25,2023-06-17T07:55:46.185000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [24]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
67,8116,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM012,2023-06-02,2023-06-17T07:58:30.050000+00:00
68,8117,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM011,2023-06-02,2023-06-17T07:58:30.050000+00:00
69,8118,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM010,2023-06-02,2023-06-17T07:58:30.050000+00:00
70,8119,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM009,2023-06-02,2023-06-17T07:58:30.050000+00:00
71,8120,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM008,2023-06-02,2023-06-17T07:58:30.050000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,84378,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES1,2022-11-03,2023-06-17T07:51:01.116000+00:00
3872,84379,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES2,2022-11-03,2023-06-17T07:51:01.116000+00:00
3873,84380,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES3,2022-11-03,2023-06-17T07:51:01.116000+00:00
3874,84381,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES4,2022-11-03,2023-06-17T07:51:01.116000+00:00


### Cohérence adresse - coordonnées
- 440 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [25]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,826,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34211,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34211,2023-06-18,2023-06-18T02:54:47.683000+00:00
1,827,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34212,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34212,2023-06-18,2023-06-18T02:54:47.683000+00:00
2,830,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34412,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34412,2023-06-18,2023-06-18T02:54:47.683000+00:00
3,831,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34411,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34411,2023-06-18,2023-06-18T02:54:47.683000+00:00
4,2331,sav@izivia.com,SOREGIES,"[0.30007, 46.54829]",RUE DE LÉCORCERIE,FRS86PSOREB1511,POITIERS SUD HPC,Voirie,1,Accès libre,24/7,false,FRS86ESOREB1511,2023-06-18,2023-06-18T02:54:43.675000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,84241,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S5,2022-11-03,2023-06-17T07:51:01.116000+00:00
3798,84242,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S6,2022-11-03,2023-06-17T07:51:01.116000+00:00
3799,84243,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE,FR55CPBP73190,BP - AIRE DE L'ABIS SAINT-JEOIRE-PRIEURE - 73190,Station dédiée à la recharge rapide,8,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S7,2022-11-03,2023-06-17T07:51:01.116000+00:00
3869,84376,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S1,2022-11-03,2023-06-17T07:51:01.116000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [26]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
113,9367,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-06-17T07:58:30.050000+00:00
114,9368,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-06-17T07:58:30.050000+00:00
115,9369,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-06-17T07:58:30.050000+00:00
116,9370,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-06-17T07:58:30.050000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [27]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2199,75176,web@freshmile.com,Freshmile,"[6.70933, 49.1065]",Place du Marché 57500 Saint-Avold,FRFR1PYKTKBS,"Saint-Avold, place du Marché",Voirie,3,Accès libre,24/7,false,FRFR1ECJQR1,2022-10-06,2023-06-17T07:54:02.990000+00:00
2200,75185,web@freshmile.com,Freshmile,"[6.70933, 49.1065]",Place du Marché 57500 Saint-Avold,FRFR1PYKTKBS,"Saint-Avold, place du Marché",Voirie,3,Accès libre,24/7,false,FRFR1ECJQR2,2022-10-06,2023-06-17T07:54:02.990000+00:00
2201,75444,web@freshmile.com,Freshmile,"[6.70929, 49.1065]",Place du Marché 57500 Saint-Avold,FRFR1PYKTKBS,"Saint-Avold, place du Marché",Voirie,3,Accès libre,24/7,false,FRFR1EGYFJ1,2022-10-06,2023-06-17T07:54:02.990000+00:00
2202,75699,web@freshmile.com,Freshmile,"[7.49739994, 47.66957673]",Allée Eugène Moser 68680 Kembs,FRFR1PKCFNXIKPD8,"Kembs, Rue De L'Europe",Voirie,2,Accès libre,24/7,false,FRFR1EWVHE1,2023-06-14,2023-06-17T07:54:02.990000+00:00
2203,75735,web@freshmile.com,Freshmile,"[7.49751204, 47.66949847]",Allée Eugène Moser 68680 Kembs,FRFR1PKCFNXIKPD8,"Kembs, Rue De L'Europe",Voirie,2,Accès libre,24/7,false,FRFR1EWVHE2,2023-06-14,2023-06-17T07:54:02.990000+00:00
2204,75840,web@freshmile.com,Freshmile,"[1.523894, 44.147449]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF1,2022-11-09,2023-06-17T07:54:02.990000+00:00
2205,75842,web@freshmile.com,Freshmile,"[-0.672947, 44.859769]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,4,Accès libre,24/7,false,FRFR1EPNZZ1,2022-11-09,2023-06-17T07:54:02.990000+00:00
2207,75861,web@freshmile.com,Freshmile,"[-1.15518, 44.6395]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,4,Accès libre,24/7,false,FRFR1EPNZZ2,2022-11-09,2023-06-17T07:54:02.990000+00:00
2210,75866,web@freshmile.com,Freshmile,"[4.636316, 44.26687]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF2,2022-11-09,2023-06-17T07:54:02.990000+00:00
2211,75869,web@freshmile.com,Freshmile,"[0.66337, 43.85402]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE2,2022-11-09,2023-06-17T07:54:02.990000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [28]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
45,5377,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ETCIX5105,2022-07-21,2023-06-17T07:58:53.797000+00:00
46,5379,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EREUO5946,2022-07-21,2023-06-17T07:58:53.797000+00:00
47,5380,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-06-17T07:58:53.797000+00:00
48,5381,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-06-17T07:58:53.797000+00:00
49,5382,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-06-17T07:58:53.797000+00:00
50,5383,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-06-17T07:58:53.797000+00:00
51,5384,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-06-17T07:58:53.797000+00:00
52,5385,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-06-17T07:58:53.797000+00:00
53,5386,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-06-17T07:58:53.797000+00:00
55,5388,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-06-17T07:58:53.797000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [29]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3490,84105,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-06-17T07:51:01.116000+00:00
3561,84197,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-06-17T07:51:01.116000+00:00
3562,84198,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-06-17T07:51:01.116000+00:00
3588,84293,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-06-17T07:51:01.116000+00:00
3611,84319,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-06-17T07:51:01.116000+00:00
3612,84320,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-06-17T07:51:01.116000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [30]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
37,4472,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-06-18,2023-06-18T02:54:38.787000+00:00
38,4473,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-06-18,2023-06-18T02:54:38.787000+00:00
39,4474,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-06-18,2023-06-18T02:54:38.787000+00:00
40,4475,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-06-18,2023-06-18T02:54:38.787000+00:00


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [31]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
64380,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [32]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
5239,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2
5240,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [33]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
82400,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
82401,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
82402,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
82403,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
82404,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [34]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
65099,65411,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-06-17T07:55:06.631000+00:00
65100,65412,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-06-17T07:55:06.631000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [35]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
64653,64958,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-06-17T07:55:10.242000+00:00
64654,64959,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-06-17T07:55:10.242000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [36]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1902', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
10044,10154,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19027,2023-06-02,2023-06-17T07:58:29.780000+00:00
10045,10155,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19026,2023-06-02,2023-06-17T07:58:29.780000+00:00
10066,10176,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19021,2023-06-02,2023-06-17T07:58:29.780000+00:00
10067,10177,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19022,2023-06-02,2023-06-17T07:58:29.780000+00:00
10068,10178,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19023,2023-06-02,2023-06-17T07:58:29.780000+00:00
10069,10179,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19024,2023-06-02,2023-06-17T07:58:29.780000+00:00
10070,10180,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19025,2023-06-02,2023-06-17T07:58:29.780000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [37]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1396,51613,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540013,2023-03-17,2023-06-17T07:55:57.156000+00:00
1397,51617,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540012,2023-03-17,2023-06-17T07:55:57.156000+00:00
1427,52778,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540014,2023-03-17,2023-06-17T07:55:57.156000+00:00
1431,52983,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540011,2023-03-17,2023-06-17T07:55:57.156000+00:00
1584,64885,support@passpasselectrique.fr,Pass Pass Electrique,"[2.460441, 50.78763]",1bis Rue De La Gare 59670 Bavinchove,FRH14E59054001,Bavinchove - Gare,Parking public,2,Accès libre,24/7,false,FRH14E59054001,2023-01-27,2023-06-17T07:55:13.011000+00:00
